# Experiment

In [ ]:
%load_ext autoreload
%autoreload 2

Beware, this will result in
<pre>
    0:00 <span style="color:#42A5F5"> 2.3K</span> <b>0_init_pipeline.pkl</b>
    0:01 <span style="color:#42A5F5">  71K</span> S37_detector_data_slices.png
    0:01 <span style="color:#42A5F5"> 110K</span> S37_detector_data_sum.png
    0:01 <span style="color:#42A5F5"> 144K</span> S37_detector_data_orthogonalisation.png
    0:01 <span style="color:#FF7043"> 1.6G</span> S37_preprocessed_data.cxi
    0:01 <span style="color:#42A5F5"> 3.3K</span> preprocess_output.log
    0:01 <span style="color:#E53935">  15G</span> <b>1_preprocess.pkl</b>
    0:12 <span style="color:#42A5F5">  53K</span> phase_retrieval_output.log
    0:13 <span style="color:#E53935">  15G</span> <b>2_phase_retrieval.pkl</b>
    0:14 <span style="color:#E53935">  15G</span> <b>3_phase_analysis.pkl</b>
    0:15 <span style="color:#FF7043"> 9.6G</span> pynx_phasing/
    0:15 <span style="color:#E53935">  15G</span> <b>4_best_candidates.pkl</b>
    0:37 <span style="color:#B0B0B0">    0</span> mode_decomposition_output.log
    0:38 <span style="color:#FFA726"> 736M</span> S37_pynx_reconstruction_mode.cxi
    0:38 <span style="color:#E53935">  15G</span> <b>5_mode_decompose.pkl</b>
    0:38 <span style="color:#B0B0B0">    0</span> postprocess_output.log
    0:39 <span style="color:#42A5F5">  78K</span> S37_amplitude_distribution.png
    0:39 <span style="color:#42A5F5"> 156K</span> S37_summary_plot.png
    0:39 <span style="color:#42A5F5"> 102K</span> S37_strain_methods.png
    0:39 <span style="color:#42A5F5">  88K</span> S37_shear_displacement.png
    0:39 <span style="color:#42A5F5"> 132K</span> S37_strain_statistics.png
    0:39 <span style="color:#42A5F5"> 203K</span> S37_3d_strain_views.png
    0:39 <span style="color:#42A5F5"> 158K</span> S37_final_object_fft.png
    0:39 <span style="color:#FFA726"> 869M</span> S37_postprocessed_data.cxi
    0:39 <span style="color:#66BB6A">  74M</span> S37_structural_properties.npz
    0:39 <span style="color:#66BB6A">  90M</span> S37_structural_properties.vti
    0:39 <span style="color:#42A5F5"> 2.3K</span> S37_parameters.yml
    0:39 <span style="color:#D32F2F">  17G</span> <b>6_postprocess.pkl</b>
</pre>

In [ ]:
import ewoks_cdiutils

In [ ]:
import os

simulation_parameters = {
    # metadata
    "beamline_setup": "p10",
    "experiment_data_dir_path": (
        #"./data/"
        "/data/projects/carine/Experiments/P10_June24/raw/"
    ),
    "sample_name": "Pd_YSZ",  
    "scan": 37,
    
    # preprocessing parameters
    "preprocess_shape": (600, 400, 400),  # define cropped window size
    "voxel_reference_methods": [(300, 1326, 1489)],  # centring method sequence
    "background_level": 2,  # background intensity level to remove
    "hot_pixel_filter": True,  # remove isolated hot pixels
    "light_loading": True,
    "det_calib_params": {
        'distance': 1.82169,        # Sample-detector distance in meters
        'cch1': 1055.74,            # Direct beam position on detector (vert. pixel)
        'cch2': 1332.94,            # Direct beam position on detector (hor. pixel)
        'pwidth1': 7.5e-05,
        'pwidth2': 7.5e-05,  
    }
}

dump_directory = "/data/id01/inhouse/polack/scratch/ewoks_results/"  # last '/' important for postprocess

In [ ]:
from ewoks import execute_graph

nodes = [
    {
        "id": "init_pipeline",
        "task_type": "class",
        "task_identifier": "ewoks_cdiutils.InitBcdiPipeline",
    },
    {
        "id": "preprocess",
        "task_type": "class",
        "task_identifier": "ewoks_cdiutils.BcdiPreprocess",
    },
    {
        "id": "phase_retrieval",
        "task_type": "class",
        "task_identifier": "ewoks_cdiutils.PhaseRetrieval",
    },
    {
        "id": "phase_analysis",
        "task_type": "class",
        "task_identifier": "ewoks_cdiutils.PhaseAnalysis",
    },
    {
        "id": "select_best",
        "task_type": "class",
        "task_identifier": "ewoks_cdiutils.SelectingBest",
    },
    {
        "id": "mode_decompose",
        "task_type": "class",
        "task_identifier": "ewoks_cdiutils.ModeDecomposition",
    },
    {
        "id": "postprocess",
        "task_type": "class",
        "task_identifier": "ewoks_cdiutils.PostProcess",
    },
    {
        "id": "debug_class",
        "task_type": "class",
        "task_identifier": "ewoks_cdiutils.DebugClass",
    },
]

links = [
    {
        "source": "init_pipeline",
        "target": "preprocess",
        "data_mapping": [{"source_output": "pipeline_path", "target_input": "pipeline_path"}],
    },
    {
        "source": "preprocess",
        "target": "phase_retrieval",
        "data_mapping": [{"source_output": "pipeline_path", "target_input": "pipeline_path"}],
    },
    {
        "source": "phase_retrieval",
        "target": "phase_analysis",
        "data_mapping": [{"source_output": "pipeline_path", "target_input": "pipeline_path"}],
    },
    {
        "source": "phase_analysis",
        "target": "select_best",
        "data_mapping": [{"source_output": "pipeline_path", "target_input": "pipeline_path"}],
    },
    {
        "source": "select_best",
        "target": "mode_decompose",
        "data_mapping": [{"source_output": "pipeline_path", "target_input": "pipeline_path"}],
    },
    {
        "source": "mode_decompose",
        "target": "postprocess",
        "data_mapping": [{"source_output": "pipeline_path", "target_input": "pipeline_path"}],
    },
    {
        "source": "postprocess",
        "target": "debug_class",
        "data_mapping": [{"source_output": "pipeline_path", "target_input": "path"}],
    },
]

workflow = {"graph": {"id": "hello_world"}, "nodes": nodes, "links": links}

inputs = [
    {"id": "init_pipeline", "name": "parameters", "value": simulation_parameters},
    {"id": "init_pipeline", "name": "output_directory", "value": dump_directory},

    {"id": "preprocess", "name": "output_directory", "value": dump_directory},
    {"id": "phase_retrieval", "name": "output_directory", "value": dump_directory},
    {"id": "phase_analysis", "name": "output_directory", "value": dump_directory},
    {"id": "select_best", "name": "output_directory", "value": dump_directory},
    {"id": "mode_decompose", "name": "output_directory", "value": dump_directory},
    {"id": "postprocess", "name": "output_directory", "value": dump_directory},
]

In [ ]:
result = execute_graph(workflow, inputs=inputs)
print(result)

## Simulating restart

In [ ]:
restarted_workflow = {
    "graph": {"id": "restarted_workflow"},
    "nodes": workflow["nodes"][5:],
    "links": workflow["links"][5:]
}

restart_file_path = os.path.join(dump_directory, "5_mode_decompose.pkl")
restarted_inputs = [
    {"id": "mode_decompose", "name": "pipeline_path", "value": restart_file_path},
    {"id": "mode_decompose", "name": "output_directory", "value": dump_directory},
    
    {"id": "postprocess", "name": "output_directory", "value": dump_directory},
]

In [ ]:
result = execute_graph(restarted_workflow, inputs=restarted_inputs)
print(result)